[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Hawksight-AI/semantica/blob/main/cookbook/advanced/05_Multi_Format_Export.ipynb)

# Advanced Multi-Format Export

## Overview

This advanced notebook demonstrates comprehensive export capabilities of Semantica's Export Module, covering all **8 export formats** plus report generation. You'll learn to export the same knowledge graph to multiple formats simultaneously, use advanced features, and leverage the method registry system.

**Documentation**: [API Reference](https://semantica.readthedocs.io/reference/export/)

### What You'll Learn

- Export knowledge graphs to all 8 supported formats
- Use exporter classes directly for fine-grained control
- Generate professional reports in multiple formats
- Work with RDF serialization, validation, and namespace management
- Register and use custom export methods
- Configure export settings programmatically
- Export vectors and embeddings for vector stores
- Export to graph databases using LPG format

### Export Formats Covered

1. **JSON/JSON-LD** - Standard JSON and JSON-LD formats
2. **RDF** - Turtle, RDF/XML, JSON-LD, N-Triples, N3
3. **CSV** - Tabular format for entities and relationships
4. **Graph Formats** - GraphML, GEXF, DOT for visualization tools
5. **OWL** - OWL/XML and Turtle for ontologies
6. **Vector** - JSON, NumPy, Binary, FAISS for vector stores
7. **LPG** - Cypher and LPG for graph databases
8. **YAML** - Semantic network and schema YAML
9. **Reports** - HTML, Markdown, JSON, Text reports

## Installation

Install Semantica from PyPI:

```bash
pip install semantica
# Or with all optional dependencies:
pip install semantica[all]
```


In [ ]:
!pip install semantica


In [ ]:
# Import core modules for building knowledge graph
from semantica.kg import GraphBuilder
from semantica.embeddings import EmbeddingGenerator
from semantica.ontology import OntologyGenerator
import os

# Create exports directory
os.makedirs("exports", exist_ok=True)


## Step 1: Create Sample Knowledge Graph and Data

Create a sample knowledge graph with entities, relationships, embeddings, and an ontology for comprehensive export demonstrations.


In [ ]:
builder = GraphBuilder()

entities = [
    {"id": "e1", "type": "Person", "name": "Alice", "properties": {"age": 30}},
    {"id": "e2", "type": "Person", "name": "Bob", "properties": {"age": 35}},
    {"id": "e3", "type": "Organization", "name": "Tech Corp", "properties": {"founded": 2010}},
]

relationships = [
    {"source": "e1", "target": "e2", "type": "knows"},
    {"source": "e1", "target": "e3", "type": "works_for"},
]

knowledge_graph = builder.build(entities + relationships)

embedding_generator = EmbeddingGenerator()
texts = [e["name"] for e in entities]
embeddings = embedding_generator.generate_embeddings(texts, data_type="text")

ontology_generator = OntologyGenerator()
ontology = ontology_generator.generate_from_graph(knowledge_graph)


## Step 2: Export to JSON

Export knowledge graph to JSON format using both the class and convenience function approaches.

**JSONExporter Features:**
- Standard JSON serialization
- JSON-LD format support with @context
- Configurable indentation
- Metadata and provenance tracking


In [ ]:
from semantica.export import JSONExporter

# Create JSON exporter with custom settings
json_exporter = JSONExporter(indent=2, include_metadata=True)

# Export to JSON format
json_exporter.export_knowledge_graph(knowledge_graph, "exports/output.json")

# Export to JSON-LD format
json_exporter.export_knowledge_graph(knowledge_graph, "exports/output.jsonld", format="json-ld")


## Step 3: Export to RDF

Export knowledge graph to multiple RDF formats (Turtle, RDF/XML, JSON-LD, N-Triples).

**RDFExporter Features:**
- Multiple RDF format support (Turtle, RDF/XML, JSON-LD, N-Triples, N3)
- Namespace management
- RDF validation
- Format conversion capabilities


In [ ]:
from semantica.export import RDFExporter, RDFSerializer, RDFValidator

# Create RDF exporter
rdf_exporter = RDFExporter()

# Export to Turtle format (human-readable)
rdf_exporter.export_knowledge_graph(knowledge_graph, "exports/output.ttl", format="turtle")

# Export to RDF/XML format
rdf_exporter.export_knowledge_graph(knowledge_graph, "exports/output.rdf", format="rdfxml")

# Export to JSON-LD format
rdf_exporter.export_knowledge_graph(knowledge_graph, "exports/output.jsonld", format="jsonld")

# Export to N-Triples format
rdf_exporter.export_knowledge_graph(knowledge_graph, "exports/output.nt", format="ntriples")

# Using RDFSerializer for format conversion
serializer = RDFSerializer()
rdf_data = serializer.convert_kg_to_rdf(knowledge_graph)
turtle_string = serializer.serialize_to_turtle(rdf_data)

# Using RDFValidator for validation
validator = RDFValidator()
validation_result = validator.validate_rdf_syntax(turtle_string, format="turtle")
print(f"RDF validation: {validation_result.get('is_valid', False)}")


## Step 4: Export to CSV

Export knowledge graph to CSV format for tabular analysis.

**CSVExporter Features:**
- Separate files for entities and relationships
- Configurable delimiter (comma, tab, semicolon)
- Automatic header generation
- Metadata serialization


In [ ]:
from semantica.export import CSVExporter

# Create CSV exporter with custom delimiter
csv_exporter = CSVExporter(delimiter=",")

# Export complete knowledge graph
csv_exporter.export_knowledge_graph(knowledge_graph, "exports/output")

# Export entities separately
entities = knowledge_graph.get("entities", [])
csv_exporter.export_entities(entities, "exports/entities.csv")

# Export relationships separately
relationships = knowledge_graph.get("relationships", [])
csv_exporter.export_relationships(relationships, "exports/relationships.csv")


## Step 5: Export to Graph Formats (GraphML, GEXF, DOT)

Export knowledge graph to graph formats for visualization tools.

**GraphExporter Features:**
- GraphML format (Cytoscape, yEd)
- GEXF format (Gephi)
- DOT format (Graphviz)
- Node and edge attribute mapping


In [ ]:
from semantica.export import GraphExporter

# Create graph exporter
graph_exporter = GraphExporter()

# Export to GraphML format (for Cytoscape, yEd)
graph_exporter.export_knowledge_graph(knowledge_graph, "exports/output.graphml", format="graphml")

# Export to GEXF format (for Gephi)
graph_exporter.export_knowledge_graph(knowledge_graph, "exports/output.gexf", format="gexf")

# Export to DOT format (for Graphviz)
graph_exporter.export_knowledge_graph(knowledge_graph, "exports/output.dot", format="dot")


## Step 6: Export to OWL

Export ontology to OWL format. **Note:** OWLExporter expects an ontology structure, not a knowledge graph.

**OWLExporter Features:**
- OWL/XML format
- OWL in Turtle format
- Class hierarchy export
- Property definition export


In [ ]:
from semantica.export import OWLExporter

# Create OWL exporter with custom URI and version
owl_exporter = OWLExporter(ontology_uri="https://example.org/ontology/", version="1.0")

# Export complete ontology to OWL/XML
owl_exporter.export(ontology, "exports/output.owl", format="owl-xml")

# Export to OWL in Turtle format
owl_exporter.export(ontology, "exports/output_owl.ttl", format="turtle")

# Export only classes
classes = ontology.get("classes", [])
owl_exporter.export_classes(classes, "exports/classes.owl")

# Export only properties
properties = ontology.get("object_properties", [])
owl_exporter.export_properties(properties, "exports/properties.owl", property_type="object")


## Step 7: Export to Vector Formats

Export vector embeddings to various formats for vector stores.

**VectorExporter Features:**
- JSON format
- NumPy format
- Binary format
- FAISS format
- Vector store integration (Weaviate, Qdrant)


In [ ]:
from semantica.export import VectorExporter

# Create vector exporter
vector_exporter = VectorExporter()

# Export to JSON format
vector_exporter.export(embeddings, "exports/output_vectors.json", format="json")

# Export to NumPy format
vector_exporter.export(embeddings, "exports/output_vectors.npy", format="numpy")

# Export to Binary format
vector_exporter.export(embeddings, "exports/output_vectors.bin", format="binary")

# Export to FAISS format
vector_exporter.export(embeddings, "exports/output_vectors.faiss", format="faiss")


## Step 8: Export to LPG (Labeled Property Graph)

Export knowledge graph to LPG format for graph databases like Neo4j and Memgraph.

**LPGExporter Features:**
- Cypher query format
- Labeled Property Graph format
- Batch node/relationship export
- Index generation


In [ ]:
from semantica.export import LPGExporter

# Create LPG exporter
lpg_exporter = LPGExporter()

# Export to Cypher format (for Neo4j, Memgraph)
lpg_exporter.export_knowledge_graph(knowledge_graph, "exports/output.cypher", format="cypher")

# Export to LPG format
lpg_exporter.export_knowledge_graph(knowledge_graph, "exports/output.lpg", format="lpg")


## Step 9: Export to YAML

Export semantic networks and schemas to YAML format.

**YAML Exporter Features:**
- Semantic network YAML export
- Schema YAML export
- Human-readable format


In [ ]:
from semantica.export import SemanticNetworkYAMLExporter, YAMLSchemaExporter

# Using SemanticNetworkYAMLExporter for knowledge graphs
yaml_exporter = SemanticNetworkYAMLExporter()
yaml_exporter.export(knowledge_graph, "exports/output_network.yaml")

# Using YAMLSchemaExporter for ontology schemas
schema_exporter = YAMLSchemaExporter()
yaml_content = schema_exporter.export_ontology_schema(ontology)
with open("exports/output_schema.yaml", "w") as f:
    f.write(yaml_content)


## Step 10: Generate Reports

Generate professional reports in multiple formats using ReportGenerator.

**ReportGenerator Features:**
- HTML reports with styling
- Markdown reports
- JSON reports
- Plain text reports
- Quality metrics aggregation


In [ ]:
from semantica.export import ReportGenerator

# Prepare report data
report_data = {
    "title": "Knowledge Graph Export Report",
    "summary": "Comprehensive export of knowledge graph to multiple formats",
    "knowledge_graph": {
        "entities": len(knowledge_graph.get("entities", [])),
        "relationships": len(knowledge_graph.get("relationships", []))
    },
    "formats_exported": ["JSON", "RDF", "CSV", "GraphML", "GEXF", "OWL", "Vector", "LPG", "YAML"],
    "export_timestamp": "2024-01-01T00:00:00Z"
}

# Create report generator
report_generator = ReportGenerator()

# Generate HTML report
report_generator.generate_report(report_data, "exports/report.html", format="html")

# Generate Markdown report
report_generator.generate_report(report_data, "exports/report.md", format="markdown")

# Generate JSON report
report_generator.generate_report(report_data, "exports/report.json", format="json")

# Generate Text report
report_generator.generate_report(report_data, "exports/report.txt", format="text")


## Step 11: Method Registry and Custom Methods

Register and use custom export methods with the MethodRegistry system.

**MethodRegistry Features:**
- Register custom export methods
- List available methods
- Get methods by name
- Unregister methods


In [ ]:
from semantica.export import MethodRegistry, method_registry, JSONExporter

# Define a custom export method
def custom_json_export(data, file_path, **kwargs):
    """Custom JSON export with additional formatting."""
    import json
    with open(file_path, 'w') as f:
        json.dump(data, f, indent=4, sort_keys=True)
    print(f"Custom export completed: {file_path}")

# Register custom method
MethodRegistry.register("json", "custom_formatted", custom_json_export)

# List all available methods
all_methods = method_registry.list_all()
print("Available methods:", all_methods)

# List methods for specific task
json_methods = method_registry.list_all("json")
print("JSON methods:", json_methods)

# Use registered method with JSONExporter
json_exporter = JSONExporter()
# The custom method can be used via the registry system
custom_method = method_registry.get("json", "custom_formatted")
if custom_method:
    custom_method(knowledge_graph, "exports/custom_output.json")


## Step 12: Configuration Management

Configure export settings using ExportConfig.

**ExportConfig Features:**
- Environment variable support
- Config file support
- Programmatic configuration
- Method-specific configuration


In [ ]:
from semantica.export import ExportConfig, export_config, JSONExporter

# Get current configuration
config = export_config.get("default")
print("Default config:", config)

# Set configuration programmatically
export_config.set("json", {"indent": 4, "include_metadata": True})
export_config.set("rdf", {"format": "turtle", "base_uri": "https://example.org/"})

# Get method-specific configuration
json_config = export_config.get_method_config("json")
print("JSON config:", json_config)

# Set method-specific configuration
export_config.set_method_config("csv", {"delimiter": "\t"})

# Use configured settings
json_exporter = JSONExporter(**export_config.get_method_config("json"))
json_exporter.export_knowledge_graph(knowledge_graph, "exports/output_configured.json")


In [ ]:
## Step 13: Verify All Exports

Verify that all exported files were created successfully.


In [ ]:
# List all exported files
export_files = [
    "exports/output.json",
    "exports/output.jsonld",
    "exports/output.ttl",
    "exports/output.rdf",
    "exports/output.nt",
    "exports/output.csv",
    "exports/entities.csv",
    "exports/relationships.csv",
    "exports/output.graphml",
    "exports/output.gexf",
    "exports/output.dot",
    "exports/output.owl",
    "exports/output_owl.ttl",
    "exports/output_vectors.json",
    "exports/output_vectors.npy",
    "exports/output.cypher",
    "exports/output_network.yaml",
    "exports/output_schema.yaml",
    "exports/report.html",
    "exports/report.md",
    "exports/report.json",
    "exports/report.txt"
]

print("📊 Export Summary:")
print("=" * 60)
for file in export_files:
    if os.path.exists(file):
        size = os.path.getsize(file)
        print(f"✅ {file:50} ({size:>10,} bytes)")
    else:
        print(f"❌ {file:50} (not found)")

print("=" * 60)
print(f"Total files checked: {len(export_files)}")
print(f"Files created: {sum(1 for f in export_files if os.path.exists(f))}")
